<a href="https://colab.research.google.com/github/AaronThomasJoseph/DL-Projects/blob/main/CS671_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import numpy as np
import tensorflow as tf
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import keras.layers
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.layers import GaussianNoise
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
#Loading Image Data
with zipfile.ZipFile("Group_21_Assignment2.zip","r") as zip_ref:
    zip_ref.extractall()
zip_ref.close()

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = plt.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

#**`Dataset`**

In [ ]:
#Train Data
train0 = np.array(load_images_from_folder("Group_21/train/0"))
train2 = np.array(load_images_from_folder("Group_21/train/2"))
train4 = np.array(load_images_from_folder("Group_21/train/4"))
train6 = np.array(load_images_from_folder("Group_21/train/6"))
train7 = np.array(load_images_from_folder("Group_21/train/7"))

train0 = train0.reshape((train0.shape[0],28*28))
train2 = train2.reshape((train2.shape[0],28*28))
train4 = train4.reshape((train4.shape[0],28*28))
train6 = train6.reshape((train6.shape[0],28*28))
train7 = train7.reshape((train7.shape[0],28*28))

x_train = np.concatenate([train0,train2,train4,train6,train7])
y_train = np.concatenate([0+np.zeros(len(train0)),2+np.zeros(len(train2)),4+np.zeros(len(train4)),
                          6+np.zeros(len(train6)),7+np.zeros(len(train7))])

#Validation Data
val0 = np.array(load_images_from_folder("Group_21/val/0"))
val2 = np.array(load_images_from_folder("Group_21/val/2"))
val4 = np.array(load_images_from_folder("Group_21/val/4"))
val6 = np.array(load_images_from_folder("Group_21/val/6"))
val7 = np.array(load_images_from_folder("Group_21/val/7"))

val0 = val0.reshape((val0.shape[0],28*28))
val2 = val2.reshape((val2.shape[0],28*28))
val4 = val4.reshape((val4.shape[0],28*28))
val6 = val6.reshape((val6.shape[0],28*28))
val7 = val7.reshape((val7.shape[0],28*28))

x_val = np.concatenate([val0,val2,val4,val6,val7])
y_val = np.concatenate([0+np.zeros(len(val0)),2+np.zeros(len(val2)),4+np.zeros(len(val4)),
                          6+np.zeros(len(val6)),7+np.zeros(len(val7))])

#Test Data
test0 = np.array(load_images_from_folder("Group_21/test/0"))
test2 = np.array(load_images_from_folder("Group_21/test/2"))
test4 = np.array(load_images_from_folder("Group_21/test/4"))
test6 = np.array(load_images_from_folder("Group_21/test/6"))
test7 = np.array(load_images_from_folder("Group_21/test/7"))

test0 = test0.reshape((test0.shape[0],28*28))
test2 = test2.reshape((test2.shape[0],28*28))
test4 = test4.reshape((test4.shape[0],28*28))
test6 = test6.reshape((test6.shape[0],28*28))
test7 = test7.reshape((test7.shape[0],28*28))

x_test = np.concatenate([test0,test2,test4,test6,test7])
y_test = np.concatenate([0+np.zeros(len(test0)),2+np.zeros(len(test2)),4+np.zeros(len(test4)),
                          6+np.zeros(len(test6)),7+np.zeros(len(test7))])

del train0,train2,train4,train6,train7
del val0,val2,val4,val6,val7
del test0,test2,test4,test6,test7

#Normalizing Input
x_train,x_val,x_test = x_train/255,x_val/255,x_test/255

#**`FCNN with 3 Hidden Layers`**

In [ ]:
#Converting one hot encoded test label to label
def ohetolabel(array):
  output = list()
  for i in range(len(array)):
    output.append(np.argmax(array[i]))
  return np.array(output)

In [ ]:
#Plot cf matrix
def cf_matrix_plot(cf_matrix,title):
  #group_names = ['True Neg','False Pos','False Neg','True Pos','True Pos','True Pos','True Pos','True Pos','True Pos']

  group_counts = ["{0:0.0f}".format(value) for value in
                  cf_matrix.flatten()]

  group_percentages = ["{0:.2%}".format(value) for value in
                      cf_matrix.flatten()/np.sum(cf_matrix)]

  labels = [f"{v1}\n{v2}\n" for v1, v2 in
            zip(group_counts,group_percentages)]

  labels = np.asarray(labels).reshape(5,5)

  ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

  ax.set_title(title);
  ax.set_xlabel('\nPredicted Image Class')
  ax.set_ylabel('Actual Image Class');

  ## Ticket labels - List must be in alphabetical order
  ax.xaxis.set_ticklabels(['0','2','4','6','7'])
  ax.yaxis.set_ticklabels(['0','2','4','6','7'])

In [ ]:
# Neural network
def clf_1_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,C,optimizer,batch_size,error_th,p,epochs=1e3):
  model = Sequential()
  model.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid'))
  model.add(Dense(C, activation='softmax'))

  es = EarlyStopping(monitor='loss',mode='min',min_delta = error_th,verbose=1,patience=p)
  if optimizer not in ["SGD","RMSProp","Adam"]:
    opt = "SGD"
  else:
    opt = optimizer

  #One Hot Encoding
  ohe = OneHotEncoder()
  y_train = ohe.fit_transform(y_train.reshape(-1,1)).toarray()
  y_val = ohe.fit_transform(y_val.reshape(-1,1)).toarray()
  y_test = ohe.fit_transform(y_test.reshape(-1,1)).toarray()

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  history = model.fit(x_train, y_train,validation_data = (x_val,y_val) ,epochs=int(epochs), batch_size=batch_size,callbacks=[es])

  y_train_pred = model.predict(x_train)
  y_val_pred   = model.predict(x_val)
  y_test_pred  = model.predict(x_test)

  y_train_pred = ohetolabel(y_train_pred)
  y_val_pred   = ohetolabel(y_val_pred)
  y_test_pred  = ohetolabel(y_test_pred)

  y_train = ohetolabel(y_train)
  y_val   = ohetolabel(y_val)
  y_test  = ohetolabel(y_test)

  print("No of Epochs =",len(history.history["loss"]))
  plt.figure(figsize=(12.5,8))
  plt.suptitle(optimizer,fontweight="bold",y=1+1e-2)
  plt.subplot(1,2,2)
  plt.plot(history.history['accuracy'],marker='.')
  plt.plot(history.history['val_accuracy'],marker='.')
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()

  plt.subplot(1,2,1)
  plt.plot(history.history['loss'],marker='.')
  plt.plot(history.history['val_loss'],marker='.')
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()
  plt.tight_layout();plt.show()

  print("Training Accuracy =\t",history.history['accuracy'][-1])
  print("Validation Accuracy =\t",history.history['val_accuracy'][-1])


  #Generate the confusion matrix
  plt.figure(figsize=(12.5,4))
  plt.suptitle(optimizer,fontweight="bold",y=1+1e-2)
  plt.subplot(1,3,1)
  cf_matrix = confusion_matrix(y_train, y_train_pred)
  cf_matrix_plot(cf_matrix,"Training Data")

  plt.subplot(1,3,2)
  cf_matrix = confusion_matrix(y_val, y_val_pred)
  cf_matrix_plot(cf_matrix,"Validation Data")

  plt.subplot(1,3,3)
  cf_matrix = confusion_matrix(y_test, y_test_pred)
  cf_matrix_plot(cf_matrix,"Test Data")
  plt.tight_layout()
  plt.show()

  print("\tTraining Data\t\t\t\tValidation Data\t\t\t\tTest Data")
  print("Accuracy:",end=" ")
  print(np.around(accuracy_score(y_train , y_train_pred),4),end="\t"*4)
  print(np.around(accuracy_score(y_val , y_val_pred),4),end="\t"*5)
  print(np.around(accuracy_score(y_test , y_test_pred),4))

  print("Precision:",end="")
  print(np.around(precision_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(precision_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(precision_score(y_test , y_test_pred,average="macro"),4))

  print("Recall:",end="   ")
  print(np.around(recall_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(recall_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(recall_score(y_test , y_test_pred,average="macro"),4))

  print("F1-Score:",end=" ")
  print(np.around(f1_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(f1_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(f1_score(y_test , y_test_pred,average="macro"),4))


In [ ]:
# Neural network
def clf_2_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,node2,C,optimizer,batch_size,error_th,p,epochs=1e3):
  model = Sequential()
  model.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid'))
  model.add(Dense(node2, activation="sigmoid"))
  model.add(Dense(C, activation='softmax'))

  es = EarlyStopping(monitor='loss',mode='min',min_delta = error_th,verbose=1,patience=p)
  if optimizer not in ["SGD","RMSProp","Adam"]:
    opt = "SGD"
  else:
    opt = optimizer

  #One Hot Encoding
  ohe = OneHotEncoder()
  y_train = ohe.fit_transform(y_train.reshape(-1,1)).toarray()
  y_val = ohe.fit_transform(y_val.reshape(-1,1)).toarray()
  y_test = ohe.fit_transform(y_test.reshape(-1,1)).toarray()

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  history = model.fit(x_train, y_train,validation_data = (x_val,y_val) ,epochs=int(epochs), batch_size=batch_size,callbacks=[es])

  y_train_pred = model.predict(x_train)
  y_val_pred   = model.predict(x_val)
  y_test_pred  = model.predict(x_test)

  y_train_pred = ohetolabel(y_train_pred)
  y_val_pred   = ohetolabel(y_val_pred)
  y_test_pred  = ohetolabel(y_test_pred)

  y_train = ohetolabel(y_train)
  y_val   = ohetolabel(y_val)
  y_test  = ohetolabel(y_test)

  print("No of Epochs =",len(history.history["loss"]))
  plt.figure(figsize=(12.5,8))
  plt.suptitle(optimizer,fontweight="bold",y=1+1e-2)
  plt.subplot(1,2,2)
  plt.plot(history.history['accuracy'],marker='.')
  plt.plot(history.history['val_accuracy'],marker='.')
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()

  plt.subplot(1,2,1)
  plt.plot(history.history['loss'],marker='.')
  plt.plot(history.history['val_loss'],marker='.')
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()
  plt.tight_layout();plt.show()

  print("Training Accuracy =\t",history.history['accuracy'][-1])
  print("Validation Accuracy =\t",history.history['val_accuracy'][-1])


  #Generate the confusion matrix
  plt.figure(figsize=(12.5,4))
  plt.suptitle(optimizer,fontweight="bold",y=1+1e-2)
  plt.subplot(1,3,1)
  cf_matrix = confusion_matrix(y_train, y_train_pred)
  cf_matrix_plot(cf_matrix,"Training Data")

  plt.subplot(1,3,2)
  cf_matrix = confusion_matrix(y_val, y_val_pred)
  cf_matrix_plot(cf_matrix,"Validation Data")

  plt.subplot(1,3,3)
  cf_matrix = confusion_matrix(y_test, y_test_pred)
  cf_matrix_plot(cf_matrix,"Test Data")
  plt.tight_layout()
  plt.show()

  print("\tTraining Data\t\t\t\tValidation Data\t\t\t\tTest Data")
  print("Accuracy:",end=" ")
  print(np.around(accuracy_score(y_train , y_train_pred),4),end="\t"*4)
  print(np.around(accuracy_score(y_val , y_val_pred),4),end="\t"*5)
  print(np.around(accuracy_score(y_test , y_test_pred),4))

  print("Precision:",end="")
  print(np.around(precision_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(precision_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(precision_score(y_test , y_test_pred,average="macro"),4))

  print("Recall:",end="   ")
  print(np.around(recall_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(recall_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(recall_score(y_test , y_test_pred,average="macro"),4))

  print("F1-Score:",end=" ")
  print(np.around(f1_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(f1_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(f1_score(y_test , y_test_pred,average="macro"),4))

In [ ]:
# Neural network
def clf_3_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e3):

  tf.keras.initializers.RandomNormal(mean=0.0, stddev=1, seed=42)
  model = Sequential()
  model.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid',
                  kernel_initializer='random_normal',bias_initializer='zeros'))
  model.add(Dense(node2, activation="sigmoid",
                  kernel_initializer='random_normal',bias_initializer='zeros'))
  model.add(Dense(node3, activation='sigmoid',
                  kernel_initializer='random_normal',bias_initializer='zeros'))
  model.add(Dense(C, activation='softmax',
                  kernel_initializer='random_normal',bias_initializer='zeros'))

  es = EarlyStopping(monitor='loss',mode='min',min_delta = error_th,verbose=1,patience=p)
  if optimizer not in ["SGD","RMSProp","Adam"]:
    opt = "SGD"
  else:
    opt = optimizer

  #One Hot Encoding
  ohe = OneHotEncoder()
  y_train = ohe.fit_transform(y_train.reshape(-1,1)).toarray()
  y_val = ohe.fit_transform(y_val.reshape(-1,1)).toarray()
  y_test = ohe.fit_transform(y_test.reshape(-1,1)).toarray()

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  history = model.fit(x_train, y_train,validation_data = (x_val,y_val) ,epochs=int(epochs), batch_size=batch_size,callbacks=[es])

  y_train_pred = model.predict(x_train)
  y_val_pred   = model.predict(x_val)
  y_test_pred  = model.predict(x_test)

  y_train_pred = ohetolabel(y_train_pred)
  y_val_pred   = ohetolabel(y_val_pred)
  y_test_pred  = ohetolabel(y_test_pred)

  y_train = ohetolabel(y_train)
  y_val   = ohetolabel(y_val)
  y_test  = ohetolabel(y_test)

  print("No of Epochs =",len(history.history["loss"]))
  plt.figure(figsize=(12.5,8))
  plt.suptitle(optimizer,fontweight="bold",y=1+1e-2)
  plt.subplot(1,2,2)
  plt.plot(history.history['accuracy'],marker='.')
  plt.plot(history.history['val_accuracy'],marker='.')
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()

  plt.subplot(1,2,1)
  plt.plot(history.history['loss'],marker='.')
  plt.plot(history.history['val_loss'],marker='.')
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()
  plt.tight_layout();plt.show()

  print("Training Accuracy =\t",history.history['accuracy'][-1])
  print("Validation Accuracy =\t",history.history['val_accuracy'][-1])


  #Generate the confusion matrix
  plt.figure(figsize=(12.5,4))
  plt.suptitle(optimizer,fontweight="bold",y=1+1e-2)
  plt.subplot(1,3,1)
  cf_matrix = confusion_matrix(y_train, y_train_pred)
  cf_matrix_plot(cf_matrix,"Training Data")

  plt.subplot(1,3,2)
  cf_matrix = confusion_matrix(y_val, y_val_pred)
  cf_matrix_plot(cf_matrix,"Validation Data")

  plt.subplot(1,3,3)
  cf_matrix = confusion_matrix(y_test, y_test_pred)
  cf_matrix_plot(cf_matrix,"Test Data")
  plt.tight_layout()
  plt.show()

  print("\tTraining Data\t\t\t\tValidation Data\t\t\t\tTest Data")
  print("Accuracy:",end=" ")
  print(np.around(accuracy_score(y_train , y_train_pred),4),end="\t"*4)
  print(np.around(accuracy_score(y_val , y_val_pred),4),end="\t"*5)
  print(np.around(accuracy_score(y_test , y_test_pred),4))

  print("Precision:",end="")
  print(np.around(precision_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(precision_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(precision_score(y_test , y_test_pred,average="macro"),4))

  print("Recall:",end="   ")
  print(np.around(recall_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(recall_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(recall_score(y_test , y_test_pred,average="macro"),4))

  print("F1-Score:",end=" ")
  print(np.around(f1_score(y_train , y_train_pred,average="macro"),4),end="\t"*4)
  print(np.around(f1_score(y_val , y_val_pred,average="macro"),4),end="\t"*5)
  print(np.around(f1_score(y_test , y_test_pred,average="macro"),4))

In [ ]:
node1 = 30
node2 = 20
node3 = 10
C = 5
error_th = 1e-4
p = 1

In [ ]:
optimizer ='SGD'
batch_size = 1
tensorflow.keras.optimizers.SGD(learning_rate=0.001)
clf_3_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)

In [ ]:
optimizer  ='SGD with momentum (NAG)'
batch_size = 32
tensorflow.keras.optimizers.SGD(learning_rate=0.001,momentum = 0.9,nesterov=True)
clf_3_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)

In [ ]:
optimizer  ='Vanilla Gradient Descent'
batch_size = len(x_train)
tensorflow.keras.optimizers.SGD(learning_rate=0.001)
clf_3_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,node2,node3,C,optimizer,batch_size,error_th,p=1000,epochs=1e5)

In [ ]:
optimizer  ='RMSprop'
batch_size = 32
tensorflow.keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.9, rho = 0.99)
clf_3_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)

In [ ]:
optimizer  ='Adam'
batch_size = 32
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
clf_3_layer(x_train,y_train,x_val,y_val,x_test,y_test,node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)

#**`AutoEncoder`**

##**`AutoEncoder with 1 Hidden Layers`**

In [ ]:
# Auto Encoder
def autoencoder_1_layer(x_train,x_val,x_test,y_train,y_val,y_test,node1,optimizer,batch_size,error_th,epochs=1e3):
  model = Sequential()
  model.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid'))
  model.add(Dense(x_train.shape[1], activation='linear'))

  es = EarlyStopping(monitor='loss',mode='min',min_delta = error_th,verbose=1,patience=1)

  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
  history = model.fit(x_train, x_train,validation_data = (x_val,x_val) ,epochs=int(epochs), batch_size=batch_size,callbacks=[es])

  print("No of Epochs =",len(history.history["loss"]))
  plt.figure(figsize=(12.5,8))
  plt.suptitle("1 Hidden Layer",fontweight="bold",y=1+1e-2)
  plt.subplot(1,1,1)
  plt.plot(history.history['loss'],marker='.')
  plt.plot(history.history['val_loss'],marker='.')
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()
  plt.tight_layout();plt.show()

  print("Training Loss\t=",history.history['loss'][-1])
  print("Validation Loss\t=",history.history['val_loss'][-1])

  x_train_pred = model.predict(x_train)
  x_val_pred   = model.predict(x_val)
  x_test_pred  = model.predict(x_test)
  test_loss = ((x_test-x_test_pred)**2).mean()
  print("Test Loss\t=",test_loss)

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Average Reconstruction Error vs Data",fontweight="bold",y=1+1e-2)
  plt.subplot(1,1,1)
  plt.bar(height = [history.history['loss'][-1],history.history['val_loss'][-1],test_loss],x = ['Training Data','Validation Data','Test Data'])
  plt.ylabel('Average Reconstruction Error')
  plt.xlabel('Data')
  plt.grid()
  plt.tight_layout();plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Training Set\nOriginal Image",fontweight="bold",y=1+1e-2)
  index = 1
  C = set(y_train)
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_train[y_train==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Training Set\nReconstructed Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_train_pred[y_train==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Validation Set\nOriginal Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_val[y_val==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Validation Set\nReconstructed Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_val_pred[y_val==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  #Weight Visualization
  plt.figure(figsize=(12.5,node1))
  plt.suptitle("Weight Vizualization",fontweight="bold",y=1+1e-2)
  weights = model.layers[0].get_weights()
  index = 1
  for i in range(node1):
    plt.subplot(node1//4,4,index)
    img = weights[0].T[i]
    img /= np.sqrt(sum(img**2))
    plt.imshow(img.reshape(28,28),cmap='gray')
    plt.title("Hidden Node "+str(int(i+1)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  #Encoder
  model2 = Sequential()
  model2.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid', weights=model.layers[0].get_weights()))

  x_train_compress = model2.predict(x_train)
  x_val_compress = model2.predict(x_val)
  x_test_compress = model2.predict(x_test)
  return x_train_compress,x_val_compress,x_test_compress

In [ ]:
node1 = 64
epochs = 1e5
error_th = 1e-4
optimizer = 'Adam'
batch_size = 32
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
x_train_compress,x_val_compress,x_test_compress = autoencoder_1_layer(x_train,x_val,x_test,y_train,y_val,y_test,node1,optimizer,batch_size,error_th,epochs)

In [ ]:
#FCNN using Adam optimizer for classification.
node1 = 30
node2 = 20
node3 = 10
C = 5
error_th = 1e-4
p = 1

optimizer  ='Adam'
batch_size = 32
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
clf_3_layer(x_train_compress,y_train,x_val_compress,y_val,x_test_compress,y_test,
            node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)

##**`AutoEncoder with 3 Hidden Layers`**

In [ ]:
# Auto Encoder
def autoencoder_3_layer(x_train,x_val,x_test,y_train,y_val,y_test,node1,node2,optimizer,batch_size,error_th,epochs=1e3):
  model = Sequential()
  model.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid'))
  model.add(Dense(node2, activation='sigmoid'))
  model.add(Dense(node1, activation='sigmoid'))
  model.add(Dense(x_train.shape[1], activation='linear'))

  es = EarlyStopping(monitor='loss',mode='min',min_delta = error_th,verbose=1,patience=1)

  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
  history = model.fit(x_train, x_train,validation_data = (x_val,x_val) ,epochs=int(epochs), batch_size=batch_size,callbacks=[es])

  print("No of Epochs =",len(history.history["loss"]))
  plt.figure(figsize=(12.5,8))
  plt.suptitle("3 Hidden Layer",fontweight="bold",y=1+1e-2)
  plt.subplot(1,1,1)
  plt.plot(history.history['loss'],marker='.')
  plt.plot(history.history['val_loss'],marker='.')
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()
  plt.tight_layout();plt.show()

  print("Training Loss\t=",history.history['loss'][-1])
  print("Validation Loss\t=",history.history['val_loss'][-1])

  x_train_pred = model.predict(x_train)
  x_val_pred   = model.predict(x_val)
  x_test_pred  = model.predict(x_test)
  test_loss = ((x_test-x_test_pred)**2).mean()
  print("Test Loss\t=",test_loss)

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Average Reconstruction Error vs Data",fontweight="bold",y=1+1e-2)
  plt.subplot(1,1,1)
  plt.bar(height = [history.history['loss'][-1],history.history['val_loss'][-1],test_loss],x = ['Training Data','Validation Data','Test Data'])
  plt.ylabel('Average Reconstruction Error')
  plt.xlabel('Data')
  plt.grid()
  plt.tight_layout();plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Training Set\nOriginal Image",fontweight="bold",y=1+1e-2)
  index = 1
  C = set(y_train)
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_train[y_train==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Training Set\nReconstructed Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_train_pred[y_train==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Validation Set\nOriginal Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_val[y_val==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Validation Set\nReconstructed Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_val_pred[y_val==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  #Weight Visualization
  plt.figure(figsize=(12.5,node2))
  plt.suptitle("Weight Vizualization",fontweight="bold",y=1+1e-2)
  weights = model.layers[1].get_weights()
  index = 1
  for i in range(node2):
    plt.subplot(node2//4,4,index)
    img = weights[0].T[i]
    img /= np.sqrt(sum(img**2))
    plt.imshow(img.reshape(16,16),cmap='gray')
    plt.title("Hidden Node "+str(int(i+1)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  #Encoder
  model2 = Sequential()
  model2.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid', weights=model.layers[0].get_weights()))
  model2.add(Dense(node2, activation='sigmoid',weights=model.layers[1].get_weights()))

  x_train_compress = model2.predict(x_train)
  x_val_compress = model2.predict(x_val)
  x_test_compress = model2.predict(x_test)
  return x_train_compress,x_val_compress,x_test_compress

In [ ]:
node1 = 256
node2 = 64
epochs = 1e5
error_th = 1e-4
optimizer = 'Adam'
batch_size = 32
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
x_train_compress,x_val_compress,x_test_compress = autoencoder_3_layer(x_train,x_val,x_test,y_train,y_val,y_test,node1,node2,optimizer,batch_size,error_th,epochs)

In [ ]:
#FCNN using Adam optimizer for classification.
node1 = 30
node2 = 20
node3 = 10
C = 5
error_th = 1e-4
p = 1

optimizer  ='Adam'
batch_size = 32
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
clf_3_layer(x_train_compress,y_train,x_val_compress,y_val,x_test_compress,y_test,
            node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)

##**`Denoising AutoEncoder with 1 Hidden Layers`**

In [ ]:
class noiseLayer(tf.keras.layers.Layer):
  def __init__(self,noise_pct,mean,std):
    super(noiseLayer, self).__init__()
    self.noise_pct = noise_pct
    self.mean = mean
    self.std  = std

  def call(self, input):
    mean = self.mean
    std  = self.std
    noise = np.random.normal(size = [784],loc = mean,scale = std)
    #if np.random.rand()*100>noise_pct:
    #  noise *= 0
    index = list(range(len(noise)))
    np.random.shuffle(index)
    noise[index[int(self.noise_pct/100*len(noise)):]] = 0
    noise = tf.convert_to_tensor(noise,dtype=np.float32)
    return input + noise

# Denoising Auto Encoder
def denoising_autoencoder_1_layer(x_train,x_val,x_test,y_train,y_val,y_test,node1,optimizer,batch_size,error_th,noise_pct,epochs=1e3):
  model = Sequential()
  model.add(noiseLayer(noise_pct,mean=0,std=1/np.sqrt(784)))
  model.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid'))
  model.add(Dense(x_train.shape[1], activation='linear'))

  es = EarlyStopping(monitor='loss',mode='min',min_delta = error_th,verbose=1,patience=1)

  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
  history = model.fit(x_train, x_train,validation_data = (x_val,x_val) ,epochs=int(epochs),
                      batch_size=batch_size,callbacks=[es])

  print("No of Epochs =",len(history.history["loss"]))
  plt.figure(figsize=(12.5,8))
  plt.suptitle("1 Hidden Layer",fontweight="bold",y=1+1e-2)
  plt.subplot(1,1,1)
  plt.plot(history.history['loss'],marker='.')
  plt.plot(history.history['val_loss'],marker='.')
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.grid()
  plt.tight_layout();plt.show()

  print("Training Loss\t=",history.history['loss'][-1])
  print("Validation Loss\t=",history.history['val_loss'][-1])

  x_train_pred = model.predict(x_train)
  x_val_pred   = model.predict(x_val)
  x_test_pred  = model.predict(x_test)
  test_loss = ((x_test-x_test_pred)**2).mean()
  print("Test Loss\t=",test_loss)

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Average Reconstruction Error vs Data",fontweight="bold",y=1+1e-2)
  plt.subplot(1,1,1)
  plt.bar(height = [history.history['loss'][-1],history.history['val_loss'][-1],test_loss],x = ['Training Data','Validation Data','Test Data'])
  plt.ylabel('Average Reconstruction Error')
  plt.xlabel('Data')
  plt.grid()
  plt.tight_layout();plt.show()


  plt.figure(figsize=(12.5,4))
  plt.suptitle("Training Set\nOriginal Image",fontweight="bold",y=1+1e-2)
  index = 1
  C = set(y_train)
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_train[y_train==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Training Set\nReconstructed Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_train_pred[y_train==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Validation Set\nOriginal Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_val[y_val==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  plt.figure(figsize=(12.5,4))
  plt.suptitle("Validation Set\nReconstructed Image",fontweight="bold",y=1+1e-2)
  index = 1
  for i in C:
    plt.subplot(1,5,index)
    plt.imshow(x_val_pred[y_val==i][0].reshape(28,28),cmap='gray')
    plt.title("Class "+str(int(i)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  #Weight Visualization
  plt.figure(figsize=(12.5,node1))
  plt.suptitle("Weight Vizualization",fontweight="bold",y=1+1e-2)
  weights = model.layers[1].get_weights()

  index = 1
  for i in range(8):#node1):
    plt.subplot(node1//4,4,index)
    img = weights[0].T[i]
    img /= np.sqrt(sum(img**2))
    plt.imshow(img.reshape(28,28),cmap='gray')
    plt.title("Hidden Node "+str(int(i+1)))
    plt.xticks([]);plt.yticks([])
    index+=1
  plt.tight_layout()
  plt.show()

  #Encoder
  model2 = Sequential()
  model.add(noiseLayer(noise_pct,mean=0,std=1/np.sqrt(784)))
  model2.add(Dense(node1, input_dim=x_train.shape[1], activation='sigmoid', weights=model.layers[1].get_weights()))

  x_train_compress = model2.predict(x_train)
  x_val_compress = model2.predict(x_val)
  x_test_compress = model2.predict(x_test)
  return x_train_compress,x_val_compress,x_test_compress

In [ ]:
node1 = 64
epochs = 1e5
error_th = 1e-4
optimizer = 'Adam'
batch_size = 32
noise_pct = 100
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
x_train_compress,x_val_compress,x_test_compress = denoising_autoencoder_1_layer(x_train,x_val,x_test,y_train,y_val,y_test,node1,optimizer,batch_size,error_th,noise_pct,epochs)

In [ ]:
#FCNN using Adam optimizer for classification.
node1 = 30
node2 = 20
node3 = 10
C = 5
error_th = 1e-4
p = 1

optimizer  ='Adam'
batch_size = 32
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
clf_3_layer(x_train_compress,y_train,x_val_compress,y_val,x_test_compress,y_test,
            node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)

In [ ]:
node1 = 64
epochs = 1e5
error_th = 1e-4
optimizer = 'Adam'
batch_size = 32
noise_pct = 40
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
x_train_compress,x_val_compress,x_test_compress = denoising_autoencoder_1_layer(x_train,x_val,x_test,y_train,y_val,y_test,node1,optimizer,batch_size,error_th,noise_pct,epochs)

In [ ]:
#FCNN using Adam optimizer for classification.
node1 = 30
node2 = 20
node3 = 10
C = 5
error_th = 1e-4
p = 1

optimizer  ='Adam'
batch_size = 32
tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
clf_3_layer(x_train_compress,y_train,x_val_compress,y_val,x_test_compress,y_test,
            node1,node2,node3,C,optimizer,batch_size,error_th,p,epochs=1e5)